<a href="https://colab.research.google.com/github/DeathXide/Blog/blob/main/nb/Qwen3_(4B)-GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Read our **[Qwen3 Guide](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Unsloth

Goal: To convert `Qwen3-4B-Base` into a reasoning model via GRPO by using OpenR1's Math dataset.

We first pre fine-tune the model to make GRPO skip trying to match formatting - this speeds GRPO up.

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Base",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-21 04:48:41 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-21 04:48:41 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.5.6: Fast Qwen3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Qwen3-4B-Base with actual GPU utilization = 69.34%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 4096

tokenizer_config.json:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.5.6 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


### GRPO chat template
Since we're using a base model, we should set a chat template. You can make your own chat template as well!
1. DeepSeek uses `<think>` and `</think>`, but this is **not** necessary - you can customize it however you like!
2. A `system_prompt` is recommended to at least guide the model's responses.

In [26]:
reasoning_start = "<start_working_out>" # Acts as <think>
reasoning_end   = "<end_working_out>"   # Acts as </think>
solution_start = "<SOLUTION>"
solution_end    = "</SOLUTION>"
system_prompt = f"""You are a product decoding assistant for CommScope Copper Jacks.
Your job is to convert vague or incomplete product descriptions into a structured format by reasoning over mappings, clues, and compatibility rules.

---

Target Output (JSON Format)


  brand: ,
  category: ,
  footprint: ,
  termination: ,
  color:


---

Valid Attribute Options

1. Footprint
[M series, SL, KJ]

2. Brand
[Systimax, Uniprise, CommScope]

3. Category
[CAT6A, CAT6, CAT5E]

4. Termination
[SL, T, No dust covers, With dust covers]

5. Color
[white, blue, orange, black, ivory, red, gray, green, yellow, violet, pro white, brown, light almond, pink]

---

Compatibility Mappings

Footprint ↔ Brand Compatibility

  Systimax: [M series],
  Uniprise: [M series, SL, KJ],
  CommScope: [SL, KJ]


Category Compatibility (by Footprint and Brand)

  (M series, Systimax): [CAT6A, CAT6, CAT5E],
  (M series, Uniprise): [CAT6A, CAT6, CAT5E],
  (SL, Uniprise): [CAT6A, CAT6],
  (SL, CommScope): [CAT5E],
  (KJ, Uniprise): [CAT6A, CAT6],
  (KJ, CommScope): [CAT5E]


Termination Mapping

  (M series, Systimax, CAT6A): [T],
  (M series, Systimax, CAT6): [T],
  (M series, Systimax, CAT5E): [T],
  (M series, Uniprise, CAT6A): [SL, T],
  (M series, Uniprise, CAT6): [SL, T],
  (M series, Uniprise, CAT5E): [No dust covers],
  (SL, Uniprise, CAT6A): [No dust covers, With dust covers],
  (SL, Uniprise, CAT6): [No dust covers, With dust covers],
  (SL, CommScope, CAT5E): [No dust covers, With dust covers],
  (KJ, Uniprise, CAT6A): [No dust covers],
  (KJ, Uniprise, CAT6): [No dust covers],
  (KJ, CommScope, CAT5E): [No dust covers]


---

Decoding Instructions
{reasoning_start}

Footprint
Use if explicitly mentioned (e.g., SL footprint).
Otherwise, infer from brand or intent:
- If high performance → prefer M series
- If value → SL
- If cost-sensitive → KJ
- If neither brand nor intent → default to M series

Brand
Use if explicitly mentioned (e.g., Systimax jack).
- If SL termination is mentioned → brand must be Uniprise
- If T-top termination is mentioned:
  - Could be either:
    - Systimax (all M series Systimax jacks use T by default)
    - Uniprise M series CAT6/CAT6A (supports both SL and T)
Infer brand based on:
- Systimax if performance-focused or default
- Uniprise if cost/value-focused or T is explicitly chosen with Uniprise

Special Rule on CommScope:
Do NOT default to CommScope just because the name is mentioned.
Only choose CommScope if both:
- CommScope is mentioned in the question, and
- The category is CAT5E
Otherwise prefer a different brand that supports the requested category.
If category is CAT5E and footprint is SL or KJ → brand must be CommScope

Category
Use if mentioned directly (e.g., Cat6A jack).
Else, infer from (footprint, brand) using the compatibility map.
If intent is:
- High performance → pick highest (CAT6A > CAT6 > CAT5E)
- Value → prefer CAT6, fallback to CAT5E
- Cost-sensitive → prefer CAT5E

Termination
Use termination mapping with (footprint, brand, category).
If only one option → use it.
If multiple:
- SL termination → select SL
- T-top → prefer T
- Dust covers → select accordingly
All Systimax jacks always use T termination.

Color
Use if explicitly mentioned.
If panel or faceplate color is given → match it.
If brand is Systimax and color not given → default to black
Otherwise, default to white

{reasoning_end}

---

You are given a description.
Think about how to decode the attributes of the given description and provide your working out.
Place it between {reasoning_start} and {reasoning_end}.
Then, provide your JSON between {solution_start} and {solution_end}.

---

Example Target Output

{reasoning_start}
Intent identified as not specified. Footprint explicitly or by default: SL. Category resolved using rules and brand capabilities: CAT6A. Brand chosen based on mapping and termination needs: Uniprise. Termination resolved as No dust covers using mapping logic. Color resolved as white based on explicit mention or brand default.
{reasoning_end}

{solution_start}

  brand: Uniprise,
  category: CAT6A,
  footprint: SL,
  termination: No dust covers,
  color: white

{solution_end}
"""

system_prompt

'You are a product decoding assistant for CommScope Copper Jacks.\nYour job is to convert vague or incomplete product descriptions into a structured format by reasoning over mappings, clues, and compatibility rules.\n\n---\n\nTarget Output (JSON Format)\n\n\n  brand: ,\n  category: ,\n  footprint: ,\n  termination: ,\n  color: \n\n\n---\n\nValid Attribute Options\n\n1. Footprint  \n[M series, SL, KJ]\n\n2. Brand  \n[Systimax, Uniprise, CommScope]\n\n3. Category  \n[CAT6A, CAT6, CAT5E]\n\n4. Termination  \n[SL, T, No dust covers, With dust covers]\n\n5. Color  \n[white, blue, orange, black, ivory, red, gray, green, yellow, violet, pro white, brown, light almond, pink]\n\n---\n\nCompatibility Mappings\n\nFootprint ↔ Brand Compatibility  \n\n  Systimax: [M series],\n  Uniprise: [M series, SL, KJ],\n  CommScope: [SL, KJ]\n\n\nCategory Compatibility (by Footprint and Brand)  \n\n  (M series, Systimax): [CAT6A, CAT6, CAT5E],\n  (M series, Uniprise): [CAT6A, CAT6, CAT5E],\n  (SL, Uniprise): [

We create a simple chat template below. Notice `add_generation_prompt` includes prepending `<start_working_out>` to guide the model to start its reasoning process.

In [27]:
chat_template = \
    "{% if messages[0]['role'] == 'system' %}"\
        "{{ messages[0]['content'] + eos_token }}"\
        "{% set loop_messages = messages[1:] %}"\
    "{% else %}"\
        "{{ '{system_prompt}' + eos_token }}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}{{ '{reasoning_start}' }}"\
    "{% endif %}"

# Replace with out specific template:
chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{system_prompt}'")\
    .replace("'{reasoning_start}'", f"'{reasoning_start}'")
tokenizer.chat_template = chat_template

Let's see how our chat template behaves on an example:

In [28]:
tokenizer.apply_chat_template([
    {
        "role": "user",
        "content": "Can you recommend a high-speed jack in SL footprint?"
    },
    {
        "role": "assistant",
        "content": f"""{reasoning_start}Intent identified as 'not specified'. Footprint explicitly or by default: 'SL'. Category resolved using rules and brand capabilities: 'CAT6A'. Brand chosen based on mapping and termination needs: 'Uniprise'. Termination resolved as 'No dust covers' using mapping logic. Color resolved as 'white' based on explicit mention or brand default.{reasoning_end}{solution_start}'brand': 'Uniprise', 'category': 'CAT6A', 'footprint': 'SL', 'termination': 'No dust covers', 'color': 'white'{solution_end}"""
    }
], tokenize=False, add_generation_prompt=True)


"You are a product decoding assistant for CommScope Copper Jacks.\nYour job is to convert vague or incomplete product descriptions into a structured format by reasoning over mappings, clues, and compatibility rules.\n\n---\n\nTarget Output (JSON Format)\n\n\n  brand: ,\n  category: ,\n  footprint: ,\n  termination: ,\n  color: \n\n\n---\n\nValid Attribute Options\n\n1. Footprint  \n[M series, SL, KJ]\n\n2. Brand  \n[Systimax, Uniprise, CommScope]\n\n3. Category  \n[CAT6A, CAT6, CAT5E]\n\n4. Termination  \n[SL, T, No dust covers, With dust covers]\n\n5. Color  \n[white, blue, orange, black, ivory, red, gray, green, yellow, violet, pro white, brown, light almond, pink]\n\n---\n\nCompatibility Mappings\n\nFootprint ↔ Brand Compatibility  \n\n  Systimax: [M series],\n  Uniprise: [M series, SL, KJ],\n  CommScope: [SL, KJ]\n\n\nCategory Compatibility (by Footprint and Brand)  \n\n  (M series, Systimax): [CAT6A, CAT6, CAT5E],\n  (M series, Uniprise): [CAT6A, CAT6, CAT5E],\n  (SL, Uniprise): [

In [45]:

import pandas as pd
import json
from datasets import Dataset

# Load your Excel file
df = pd.read_csv("Updated_Dataset_with_Text_Column.csv")
dataset = df.dropna(subset=['question', 'json_body_new','reasoning_new'])

# # Convert each row into a conversation list (multi-turn if needed)
# def row_to_convo(row):
#     try:
#         assistant_response = json.loads(row['json_body_new'])
#     except:
#         assistant_response = row['json_body_new']
#     return [
#         {"role": "user", "content": row['question']},
#         {"role": "assistant", "content": json.dumps(assistant_response)}
#     ]

# # Build dataset
# convos = [row_to_convo(row) for _, row in df.iterrows()]
# hf_format_data = [{"conversations": c} for c in convos]

# # Convert to HuggingFace Dataset
# dataset = Dataset.from_list(hf_format_data)

### Pre fine-tuning for formatting
We now use a subset of NVIDIA's [Open Math Reasoning dataset](https://huggingface.co/datasets/nvidia/OpenMathReasoning) which was filtered to only include high quality DeepSeek R1 traces.

We'll only filter ~59 or so examples to first "prime" / pre fine-tune the model to understand our custom GRPO formatting.

In [36]:
from datasets import load_dataset
import pandas as pd
import numpy as np

dataset = load_dataset("unsloth/OpenMathReasoning-mini", split = "cot")
dataset = dataset.to_pandas()[
    ["expected_answer", "problem", "generated_solution"]
]

# Try converting to number - if not, replace with NaN
is_number = pd.to_numeric(pd.Series(dataset["expected_answer"]), errors = "coerce").notnull()
# Select only numbers
dataset = dataset.iloc[np.where(is_number)[0]]

dataset

README.md:   0%|          | 0.00/603 [00:00<?, ?B/s]

data/cot-00000-of-00001.parquet:   0%|          | 0.00/106M [00:00<?, ?B/s]

KeyboardInterrupt: 

We have to format the dataset to follow our GRPO style formatting:

In [46]:
def format_dataset(x):
    expected_answer = x["json_body_new"]
    problem = x["question"]

    # Remove generated <think> and </think>
    thoughts = x["reasoning_new"]
    thoughts = thoughts.replace("<think>", "").replace("</think>", "")

    # Strip newlines on left and right
    thoughts = thoughts.strip()
    # Add our custom formatting
    final_prompt = \
        reasoning_start + thoughts + reasoning_end + \
        solution_start + expected_answer + solution_end
    return [
        {"role" : "system",    "content" : system_prompt},
        {"role" : "user",      "content" : problem},
        {"role" : "assistant", "content" : final_prompt},
    ]

dataset["Messages"] = dataset.apply(format_dataset, axis = 1)

Check to see if it worked:

In [47]:
tokenizer.apply_chat_template(dataset["Messages"][0], tokenize = False)

"You are a product decoding assistant for CommScope Copper Jacks.\nYour job is to convert vague or incomplete product descriptions into a structured format by reasoning over mappings, clues, and compatibility rules.\n\n---\n\nTarget Output (JSON Format)\n\n\n  brand: ,\n  category: ,\n  footprint: ,\n  termination: ,\n  color: \n\n\n---\n\nValid Attribute Options\n\n1. Footprint  \n[M series, SL, KJ]\n\n2. Brand  \n[Systimax, Uniprise, CommScope]\n\n3. Category  \n[CAT6A, CAT6, CAT5E]\n\n4. Termination  \n[SL, T, No dust covers, With dust covers]\n\n5. Color  \n[white, blue, orange, black, ivory, red, gray, green, yellow, violet, pro white, brown, light almond, pink]\n\n---\n\nCompatibility Mappings\n\nFootprint ↔ Brand Compatibility  \n\n  Systimax: [M series],\n  Uniprise: [M series, SL, KJ],\n  CommScope: [SL, KJ]\n\n\nCategory Compatibility (by Footprint and Brand)  \n\n  (M series, Systimax): [CAT6A, CAT6, CAT5E],\n  (M series, Uniprise): [CAT6A, CAT6, CAT5E],\n  (SL, Uniprise): [

Let's truncate the pre fine-tuning dataset to `max_seq_length/2` since we don't want too long reasoning traces.

Note this might take 2 minutes!

In [48]:
dataset["N"] = dataset["Messages"].apply(lambda x: len(tokenizer.apply_chat_template(x)))

dataset = dataset.loc[dataset["N"] <= max_seq_length/2].copy()
dataset.shape

(803, 12)

We then tokenize the messages and convert it to a Hugging Face compatible dataset format:

In [49]:
from datasets import Dataset

dataset["text"] = tokenizer.apply_chat_template(dataset["Messages"].values.tolist(), tokenize = False)
dataset = Dataset.from_pandas(dataset)
dataset

Dataset({
    features: ['question', 'reasoning', 'json_body', 'Matched PartNumber', 'Highlight ', 'Reviewer initials  and comments', 'json_body_new', 'reasoning_new', 'question_lower', 'text', 'Messages', 'N', '__index_level_0__'],
    num_rows: 803
})

Let's now pre fine-tune the model so it follows our custom GRPO formatting!

In [50]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 2, # Set this for 1 full training run.
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/803 [00:00<?, ? examples/s]

In [51]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 803 | Num Epochs = 2 | Total steps = 1,606
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 66,060,288/4,088,528,384 (1.62% trained)


Step,Training Loss
5,1.322500
10,0.562200
15,0.059400
20,0.038500
25,0.021800
30,0.021900
35,0.015200
40,0.021600
45,0.014300
50,0.009500


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=1606, training_loss=0.013470787208777844, metrics={'train_runtime': 3354.3679, 'train_samples_per_second': 0.479, 'train_steps_per_second': 0.479, 'total_flos': 4.65794492879831e+16, 'train_loss': 0.013470787208777844})

Let's check if the model has learnt to follow the custom format:

In [53]:
text = tokenizer.apply_chat_template(
    dataset[0]["Messages"][:2],
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 0,
    max_new_tokens = 1024,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

You are a product decoding assistant for CommScope Copper Jacks.
Your job is to convert vague or incomplete product descriptions into a structured format by reasoning over mappings, clues, and compatibility rules.

---

Target Output (JSON Format)


  brand: ,
  category: ,
  footprint: ,
  termination: ,
  color: 


---

Valid Attribute Options

1. Footprint  
[M series, SL, KJ]

2. Brand  
[Systimax, Uniprise, CommScope]

3. Category  
[CAT6A, CAT6, CAT5E]

4. Termination  
[SL, T, No dust covers, With dust covers]

5. Color  
[white, blue, orange, black, ivory, red, gray, green, yellow, violet, pro white, brown, light almond, pink]

---

Compatibility Mappings

Footprint ↔ Brand Compatibility  

  Systimax: [M series],
  Uniprise: [M series, SL, KJ],
  CommScope: [SL, KJ]


Category Compatibility (by Footprint and Brand)  

  (M series, Systimax): [CAT6A, CAT6, CAT5E],
  (M series, Uniprise): [CAT6A, CAT6, CAT5E],
  (SL, Uniprise): [CAT6A, CAT6],
  (SL, CommScope): [CAT5E],
  (KJ, U

Yes it did follow the formatting! Great! Let's remove some items before the GRPO step

In [ ]:
del dataset
torch.cuda.empty_cache()
import gc
gc.collect()

### Data Prep
<a name="Data"></a>

We're using Hugging Face's [Open R1 Math dataset](https://huggingface.co/datasets/open-r1/DAPO-Math-17k-Processed). You can also utilize OpenAI's famous [GSM8K dataset](https://huggingface.co/datasets/openai/gsm8k)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("open-r1/DAPO-Math-17k-Processed", "en", split = "train")
dataset

Let's look at the first row:

In [ ]:
['question', 'json_body_new','reasoning_new']

In [ ]:
dataset[0]["json_body_new"]

In GSM8K, ee notice all answers like about have a ####, so we extract it. But for the Open R1 dataset, we can skip the below.

In [ ]:
def extract_hash_answer(text):
    # if "####" not in text: return None
    # return text.split("####")[1].strip()
    return text
extract_hash_answer(dataset[0]["solution"])

Let's map the dataset! and see the first row:

In [54]:
dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": x["question"]},
    ],
    "answer": x["json_body_new"],
})
dataset[0]

Map:   0%|          | 0/803 [00:00<?, ? examples/s]

{'question': 'Please quote a cost-conscious Cat6 outlet in M series footprint?',
 'reasoning': '1. Provided attributes: footprint = M series, category = Cat6.\n2. Restricted options based on provided attributes: on M series, available brands for Cat6 are Systimax, Uniprise.\n3. Default selection based on priorities and reason:\n   - Footprint: M series (as specified by the customer)\n   - Brand: Systimax (global flagship brand)\n   - Category: Cat6 (as specified)\n4. Color assignment: White (default when unspecified)\n',
 'json_body': '{"brand": "Systimax", "category": "Cat6", "footprint": "M series", "color": "White", "termination": ""}',
 'Matched PartNumber': '700206725',
 'Highlight ': None,
 'Reviewer initials  and comments': 'JP:this was not right listing Systimax',
 'json_body_new': "{'brand': 'Uniprise', 'category': 'CAT6', 'footprint': 'M series', 'termination': 'SL', 'color': 'white'}",
 'reasoning_new': "Intent identified as 'cost-sensitive'. Footprint explicitly or by defau

We create a regex format to match the reasoning sections and answers:

In [55]:
import re

solution_end_regex = r"</SOLUTION>[\s]{0,}" + "(?:" + re.escape(tokenizer.eos_token) + ")?"

# Match everything from <start_working_out> to </end_working_out>, then capture solution
match_format = re.compile(
    rf"{re.escape(reasoning_start)}.*?{re.escape(reasoning_end)}"      # Match reasoning block
    rf".*?{re.escape(solution_start)}(.+?){solution_end_regex}"       # Capture solution block
    rf"[\s]*$",
    flags=re.MULTILINE | re.DOTALL
)

We verify it works:

In [ ]:
match_format.findall(
    "Let me think!<end_working_out>"\
    f"<SOLUTION>\n2\n</SOLUTION>",
)

In [58]:
text = "<start_working_out>Intent identified as 'cost-sensitive'. Footprint explicitly or by default: 'M series'. Category resolved using rules and brand capabilities: 'CAT6'. Brand chosen based on mapping and termination needs: 'Uniprise'. Termination resolved as 'SL' using mapping logic. Color resolved as 'white' based on explicit mention or brand default.<end_working_out><SOLUTION>{'brand': 'Uniprise', 'category': 'CAT6', 'footprint': 'M series', 'termination': 'SL', 'color': 'white'}</SOLUTION>\n\n"
match = match_format.findall(text)
print(match[0] if match else "No match")


{'brand': 'Uniprise', 'category': 'CAT6', 'footprint': 'M series', 'termination': 'SL', 'color': 'white'}


We now want to create a reward function to match the format exactly - we reward it with 3 points if it succeeds:

In [59]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]  # Adjust if input format differs

        # Full match using compiled regex
        if match_format.search(response):
            score += 3.0

        scores.append(score)
    return scores


If it fails, we want to reward the model if it at least follows the format partially, by counting each symbol:

In [60]:
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"].strip()  # Ensure clean input

        # Do not reward <start_working_out> as it's always prepended
        score += 0.5 if response.count(reasoning_end)   == 1 else -1.0
        score += 0.5 if response.count(solution_start)  == 1 else -1.0
        score += 0.5 if response.count(solution_end)    == 1 else -1.0

        scores.append(score)
    return scores


Finally, we want to extract the generated answer, and reward or penalize it! We also reward it based on how close the answer is to the true one via ratios:

In [61]:
import ast  # For safely parsing string to dict

def check_answer(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_format.search(r)) is not None else None
        for r in responses
    ]

    scores = []
    for guess_str, true_dict in zip(extracted_responses, answer):
        score = 0
        if guess_str is None:
            scores.append(-2.0)
            continue
        try:
            # Convert string to dictionary
            guess_dict = ast.literal_eval(guess_str.strip())

            # Full match
            if guess_dict == true_dict:
                score += 5.0
            else:
                # Partial match scoring
                matches = sum(guess_dict.get(k) == v for k, v in true_dict.items())
                total = len(true_dict)
                match_ratio = matches / total

                if match_ratio == 1.0:
                    score += 5.0
                elif match_ratio >= 0.8:
                    score += 3.5
                elif match_ratio >= 0.6:
                    score += 2.0
                elif match_ratio >= 0.4:
                    score += 1.0
                else:
                    score -= 2.0
        except Exception:
            score -= 4.5  # Malformed dictionary, bad guess

        scores.append(score)
    return scores


Also sometimes it might not be 1 number as the answer, but like a sentence for example "The solution is $20" -> we extract 20.

We also remove possible commas for example as in 123,456

In [64]:
extract_solution_dict  = re.compile(
    rf"{re.escape(solution_start)}\s*(.+?)\s*{re.escape(solution_end)}",
    flags=re.DOTALL
)
print(extract_solution_dict .findall("<SOLUTION>{'brand': 'Uniprise', 'category': 'CAT6', 'footprint': 'M series', 'termination': 'SL', 'color': 'white'}</SOLUTION>"))


["{'brand': 'Uniprise', 'category': 'CAT6', 'footprint': 'M series', 'termination': 'SL', 'color': 'white'}"]


We now prepare our main function which will print out the generated responses and the true answer, along with another reward function which converts text to float via `float` and sees if it's the same.

In [78]:
import re
import ast

# Global counters for printing debug info every few steps
global PRINTED_TIMES
PRINTED_TIMES = 0
global PRINT_EVERY_STEPS
PRINT_EVERY_STEPS = 5

# Regex to extract dictionary from <SOLUTION>...</SOLUTION>
extract_solution_dict = re.compile(
    r"<SOLUTION>\s*(\{.*?\})\s*</SOLUTION>",
    flags=re.DOTALL
)

def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        match.group(1) if (match := extract_solution_dict.search(r)) else None
        for r in responses
    ]

    scores = []
    global PRINTED_TIMES, PRINT_EVERY_STEPS

    # Optional debug printing
    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0:
        print(
            '*'*20 + f"\nQuestion:\n{question}",
            f"\nExpected Answer:\n{answer[0]}",
            f"\nModel Response:\n{responses[0]}",
            f"\nExtracted Solution:\n{extracted_responses[0]}"
        )
    PRINTED_TIMES += 1

    # Compare each extracted dict with the true one
    for guess_str, true_answer in zip(extracted_responses, answer):
        if guess_str is None:
            scores.append(-2.5)
            continue

        try:
            predicted_dict = ast.literal_eval(guess_str.strip())
            if isinstance(true_answer, str):
                true_dict = ast.literal_eval(true_answer.strip())
            else:
                true_dict = true_answer  # already a dict
        except Exception:
            scores.append(-3.0)  # Unparsable dict
            continue

        if predicted_dict == true_dict:
            scores.append(5.0)  # Perfect match
        else:
            # Partial scoring based on key matches
            correct_fields = sum(predicted_dict.get(k) == v for k, v in true_dict.items())
            total_fields = len(true_dict)
            match_ratio = correct_fields / total_fields

            if match_ratio >= 0.8:
                scores.append(3.5)
            elif match_ratio >= 0.6:
                scores.append(2.0)
            elif match_ratio >= 0.4:
                scores.append(1.0)
            else:
                scores.append(-1.5)
  # Too many mismatches
    return scores


In [79]:
dataset[0]["answer"]

"{'brand': 'Uniprise', 'category': 'CAT6', 'footprint': 'M series', 'termination': 'SL', 'color': 'white'}"

Get the top 90% prompt length so we don't accidentally truncate them!

Ie we'll remove the top 10% long prompts.

In [80]:
tokenized = dataset.map(
    lambda x: {"tokens" : tokenizer.apply_chat_template(x["prompt"], add_generation_prompt = True, tokenize = True)},
    batched = True,
)
print(tokenizer.decode(tokenized[0]["tokens"]))
tokenized = tokenized.map(lambda x: {"L" : len(x["tokens"])})

import numpy as np
maximum_length = int(np.quantile(tokenized["L"], 0.9))
print("Max Length = ", maximum_length)

# Filter only samples smaller than 90% max length
dataset = dataset.select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
del tokenized

Map:   0%|          | 0/751 [00:00<?, ? examples/s]

You are a product decoding assistant for CommScope Copper Jacks.
Your job is to convert vague or incomplete product descriptions into a structured format by reasoning over mappings, clues, and compatibility rules.

---

Target Output (JSON Format)


  brand: ,
  category: ,
  footprint: ,
  termination: ,
  color: 


---

Valid Attribute Options

1. Footprint  
[M series, SL, KJ]

2. Brand  
[Systimax, Uniprise, CommScope]

3. Category  
[CAT6A, CAT6, CAT5E]

4. Termination  
[SL, T, No dust covers, With dust covers]

5. Color  
[white, blue, orange, black, ivory, red, gray, green, yellow, violet, pro white, brown, light almond, pink]

---

Compatibility Mappings

Footprint ↔ Brand Compatibility  

  Systimax: [M series],
  Uniprise: [M series, SL, KJ],
  CommScope: [SL, KJ]


Category Compatibility (by Footprint and Brand)  

  (M series, Systimax): [CAT6A, CAT6, CAT5E],
  (M series, Uniprise): [CAT6A, CAT6, CAT5E],
  (SL, Uniprise): [CAT6A, CAT6],
  (SL, CommScope): [CAT5E],
  (KJ, U

Map:   0%|          | 0/751 [00:00<?, ? examples/s]

Max Length =  1190


<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [81]:
max_prompt_length = maximum_length + 1 # + 1 just in case!
max_completion_length = max_seq_length - max_prompt_length

from vllm import SamplingParams
vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    vllm_sampling_params = vllm_sampling_params,
    temperature = 1.0,
    learning_rate = 5e-6,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 100,
    save_steps = 100,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",

    # For optional training + evaluation
    # fp16_full_eval = True,
    # per_device_eval_batch_size = 4,
    # eval_accumulation_steps = 1,
    # eval_strategy = "steps",
    # eval_steps = 1,
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
# For optional training + evaluation
# new_dataset = dataset.train_test_split(test_size = 0.01)

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers,
    ],
    args = training_args,
    train_dataset = dataset,

    # For optional training + evaluation
    # train_dataset = new_dataset["train"],
    # eval_dataset = new_dataset["test"],
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 677 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 66,060,288/4,088,528,384 (1.62% trained)


********************
Question:
I need a SL Uniprise Cat6 jack. 
Expected Answer:
{'brand': 'Uniprise', 'category': 'CAT6', 'footprint': 'SL', 'termination': 'No dust covers', 'color': 'white'} 
Model Response:
Intent identified as 'not specified'. Footprint explicitly or by default: 'SL'. Category resolved using rules and brand capabilities: 'CAT6'. Brand chosen based on mapping and termination needs: 'Uniprise'. Termination resolved as 'No dust covers' using mapping logic. Color resolved as 'white' based on explicit mention or brand default.<end_working_out><SOLUTION>{'brand': 'Uniprise', 'category': 'CAT6', 'footprint': 'SL', 'termination': 'No dust covers', 'color': 'white'}</SOLUTION> 
Extracted Solution:
{'brand': 'Uniprise', 'category': 'CAT6', 'footprint': 'SL', 'termination': 'No dust covers', 'color': 'white'}


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / match_format_exactly,rewards / match_format_approximately,rewards / check_answer,rewards / check_numbers
1,0.006400,4.500000,0.000000,118.000000,0.160314,0.000000,1.500000,-2.000000,5.000000
2,0.006800,4.500000,0.000000,118.000000,0.170238,0.000000,1.500000,-2.000000,5.000000
3,0.009000,4.500000,0.000000,118.000000,0.224784,0.000000,1.500000,-2.000000,5.000000
4,0.007500,4.500000,0.000000,118.000000,0.188113,0.000000,1.500000,-2.000000,5.000000
5,0.007400,4.500000,0.000000,120.000000,0.185500,0.000000,1.500000,-2.000000,5.000000
6,0.008900,4.500000,0.000000,119.000000,0.222885,0.000000,1.500000,-2.000000,5.000000
7,0.011400,4.500000,0.000000,115.000000,0.283880,0.000000,1.500000,-2.000000,5.000000
8,0.006800,4.500000,0.000000,118.000000,0.169855,0.000000,1.500000,-2.000000,5.000000
9,0.008100,4.500000,0.000000,116.000000,0.203308,0.000000,1.500000,-2.000000,5.000000
10,0.006400,4.500000,0.000000,118.000000,0.159421,0.000000,1.500000,-2.000000,5.000000


********************
Question:
I need a SL Cat5e jack in Orange. 
Expected Answer:
{'brand': 'CommScope', 'category': 'CAT5E', 'footprint': 'SL', 'termination': 'No dust covers', 'color': 'orange'} 
Model Response:
Intent identified as 'not specified'. Footprint explicitly or by default: 'SL'. Category resolved using rules and brand capabilities: 'CAT5E'. Brand chosen based on mapping and termination needs: 'Uniprise'. Termination resolved as 'No dust covers' using mapping logic. Color resolved as 'orange' based on explicit mention or brand default.<end_working_out><SOLUTION>{'brand': 'CommScope', 'category': 'CAT5E', 'footprint': 'SL', 'termination': 'No dust covers', 'color': 'orange'}</SOLUTION> 
Extracted Solution:
{'brand': 'CommScope', 'category': 'CAT5E', 'footprint': 'SL', 'termination': 'No dust covers', 'color': 'orange'}
********************
Question:
Please send a Orange Systimax Cat5e jack. 
Expected Answer:
{'brand': 'Systimax', 'category': 'CAT5E', 'footprint': 'M series

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = "What is the sqrt of 101?"

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Verify LoRA is actually trained!

In [ ]:
from safetensors import safe_open

tensors = {}
with safe_open("grpo_saved_lora/adapter_model.safetensors", framework = "pt") as f:
    # Verify both A and B are non zero
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

Now we load the LoRA and test:

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 2048,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
